# Cvičení 3.5

Tématem tohoto cvičení jsou přímé metody řešení soustav lineárních rovnic. Budeme se zabývat LU rozkladem s řádkovými permutacemi.

# LU rozklad s částečnou pivotizací

Algoritmus LU rozkladu s částečnou pivotizací lze popsat např. následujícím pseudokódem.

```
U = A, L = I, P = I
for k = 1 to m-1 do
    Select i >= k maximizing abs(U(i, k))
    Swap U(k, k:m) <--> U(i, k:m)
    Swap L(k, 1:k-1) <--> L(i, 1:k-1)
    Swap P(k, :) <--> P(i, :)
    for j = k+1 to m do
        L(j, k) = U(j, k)/U(k, k)
        U(j, k:m) = U(j, k:m) - L(j, k)U(k, k:m)
    end for
end for
```

**Pozor**: značení `k:m` zde znamená sloupce od `k` po `m` *včetně*.

Na základě tohoto pseudokódu doplňte chybějící části následují funkce.

In [8]:
# ÚKOL: Doplňte následující kód

import numpy as np

def lu_decomposition_p(A):
    """
    Provádí LU dekompozici s částečnou pivotizací: PA = LU
    Použití: L, U, P = lu_decomposition_p(A)
    """
    A = np.array(A, dtype=float)
    m, n = A.shape

    L = np.eye(n, dtype=float)
    U = A.copy()
    P = np.eye(n, dtype=float)

    # LU rozklad s částečnou pivotizací
    for k in range(n-1):
        # Najděme index prvku s největší absolutní hodnotou v aktuálním sloupci
        i_max = np.argmax(np.abs(U[k:n, k])) + k

        # Prohoďme řádky k a i_max
        U[[k, i_max], k:n] = U[[i_max, k], k:n]
        L[[k, i_max], :k] = L[[i_max, k], :k]
        P[[k, i_max], :] = P[[i_max, k], :]

        # Určeme multiplikátor a updatujme L a U
        L[k+1:n, k] = U[k+1:n, k] / U[k, k]
        for j in range(k+1, n):
            U[j, k:n] -= L[j, k] * U[k, k:n]

    return L, U, P

In [73]:
# Vytvoříme náhodnou matici 5x5 a zavoláme vaši funkci
A = np.random.rand(5, 5)
L, U, P = lu_decomposition_p(A)

In [ ]:
# Otestujeme, že funkce vrací správný rozklad
np.set_printoptions(precision=3)
LU = L@U
print(LU)
print(P@A)

Známe-li rozklad matice $A$ ve tvaru
$$\mathsf{L}\mathsf{U} = \mathsf{P}\mathsf{A},$$
můžeme jej využít k řešení soustavy $\mathsf{A}\mathbf{x} = \mathbf{b}$. Soustavu upravíme na tvar 
$$\mathsf{P}\mathsf{A}\mathbf{x} = \mathsf{P}\mathbf{b}$$
a dosadíme
$$\mathsf{L}\mathsf{U}\mathbf{x}=\mathsf{P}\mathbf{b}.$$

V následující části využijte rozklad matice A a vaše metody `fsubst` a `bsubst` pro dopřednou a zpětnou substituci k vyřešení soustavy. 

In [9]:
# ÚKOL: Zkopírujte z minulého cvičení

def fsubst(L, b):
    m, n = np.shape(L)
    x = np.zeros(n)

    x[0] = b[0] / L[0, 0]

    for i in range(1, m):
        suma = 0
        for j in range(0, i):
            suma += L[i, j] * x[j]

        x[i] = ( b[i] - suma ) / L[i, i]

    return x

In [10]:
# ÚKOL: Zkopírujte z minulého cvičení

def bsubst(U, b):

    m, n = np.shape(U)
    x = np.zeros(n)

    x[n-1] = b[n-1]/U[n-1, n-1]

    for i in range(n-2, -1, -1):
        suma = 0
        for j in range(i+1, n):
            suma += U[i, j] * x[j]

        x[i] = (b[i] - suma) / U[i, i]

    return x

In [25]:
# ÚKOL: Vytvořte funkci random_system, která bude mít na vstupu dimenzi m matice A a vrátí náhodnou
# matici o rozměrech m x m a náhodný vektor pravé strany b délky m

def random_system(m):
    A = np.random.rand(m, m)
    b = np.random.rand(m)

    return A, b

In [26]:
# Použijeme vaši metodu k vytvoření náhodné soustavy Ax=b o 5 neznámých
A, b = random_system(5)

In [27]:
# Rozložíme matici A pomocí vaší implementace LU rozkladu s pivotizací
L, U, P = lu_decomposition_p(A)

In [28]:
# Permutujme vektor pravé strany
pb = P@b

# Vyřešme Ly = pb dopřednou substitucí
y = fsubst(L, pb)

# Vyřešme Ux = y zpětnou substitucí
x = bsubst(U, y)

In [ ]:
# Porovnáme vaše řešení s numpy výsledkem

x_numpy = np.linalg.solve(A, b)
print(x_numpy)
print(x)

# LDLT rozklad

LDLT rozklad je vhodný pro symetrické matice. Lze jej popsat následujícím pseudokódem:

```
n = size(A)
L = I, D = 0

v(1) = A(1, 1)
D(1, 1) = v(1)
L(2:n, 1) = A(2:n, 1)/v(1)

for j = 2 to n do
    for i = 1 to j - 1 do
        v(i) = L(j, i)D(i, i)
    end

    v(j) = A(j, j) - L(j, 1:j-1)v(1, j-1)
    D(j, j) = v(j)
    L(j+1:n, j) = (A(j+1:n, j) - L(j+1:n, 1:j-1)v(1:j-1)) / v(j)
end
```

V následující části implementujete LDLT rozklad a využijete ho k řešení soustavy se symetrickou maticí.

In [1]:
# ÚKOL: Doplňte následující kód

import numpy as np

def my_ldlt(A):
    """
    Vypočítá LDLT rozklad symetrické matice.
    Vstup: A - A symetrická matice
    Výstup: L, D - Matice dekompozice takové, že platí A = L*D*L.T
    """
    m, n = A.shape

    # Zkontrolujme, že je matice čtvercová
    if m != n:
        raise ValueError('The matrix must be square!')

    L = np.eye(m)
    D = np.zeros((m, m))

    v = np.zeros(m)
    v[0] = A[0, 0]
    D[0, 0] = v[0]
    L[1:n, 0] = A[1:n, 0] / v[0]

    for j in range(1, n):
        for i in range(j):
            v[i] = L[j, i] * D[i, i]

        v[j] = A[j, j] - np.dot(L[j, 0:j], v[0:j])
        D[j, j] = v[j]
        if j+1 <= n-1:  # Abychom v poslední iteraci nepřesáhli meze
            L[j+1:n, j] = (A[j+1:n, j] - np.dot(L[j+1:n, 0:j], v[0:j])) / v[j]

    return L, D

In [ ]:
# Vygenerujeme náhodnou symetrickou matici
A = np.random.rand(5, 5)
A = A + A.transpose()
print(A)

In [ ]:
# Otestujeme, že váš kód vrací správný výsledek
L, D = my_ldlt(A)
AA = L@D@L.transpose()
print(AA-A)

Známe-li rozklad symetrické matice $\mathsf{A} = \mathsf{L}\mathsf{D}\mathsf{L}^T$, můžeme jej využít k řešení soustavy $\mathsf{A}\mathbf{x} = \mathbf{b}$, kterou převedeme na 
$$\mathsf{L}\mathsf{D}\mathsf{L}^T\mathbf{x} = \mathbf{b}$$
a vyřešíme vhodnými substitucemi s využitím dopředné a zpětné substituce.

Rozmyslete si, jak takové substituce budou vypadat a v následující části vyřešte náhodnou soustavu.

In [15]:
# ÚKOL: Doplňte následující buňku a vyřešte soustavu rovnic Ax = b se symetrickou náhodnou maticí A a
# náhodným vektorem pravé strany b. Využijte vaše metody fsubst a bsubst
b = np.random.rand(5)
y = fsubst(L, b)

z = np.zeros(b.size)
for i in range(b.size):
    z[i] = y[i] / D[i, i]

vase_x = bsubst(L.transpose(), z)

In [ ]:
# Porovnáme vaše řešení a řešen pomocí numpy
print(vase_x)
print(np.linalg.solve(A, b))